## 4. 한국 시장 특화 부도 패턴

### 4.1 업종별 부도율 분석

In [239]:
# 업종 관련 컬럼 찾기 및 한국표준산업분류(KSIC) 매핑
industry_cols = [col for col in df.columns if '업종' in col]

if industry_cols and target_col in df.columns:
    industry_col = industry_cols[0]
    
    print("\n" + "="*80)
    print("📊 한국표준산업분류(KSIC) 데이터 로딩 및 매핑")
    print("="*80)
    
    # KSIC 파일 읽기
    ksic_file = '../data/[붙임3-3] 한국표준산업분류(10차)_표.xlsx'
    
    try:
        # 엑셀 파일의 모든 시트 확인
        xl_file = pd.ExcelFile(ksic_file)
        print(f"✅ KSIC 파일 발견. 시트 목록: {xl_file.sheet_names}")
        
        # 첫 번째 시트 읽기 (보통 여기에 분류표가 있음)
        ksic_df = pd.read_excel(ksic_file, sheet_name=0)
        print(f"✅ KSIC 데이터 로딩 완료: {ksic_df.shape}")
        print(f"   컬럼: {list(ksic_df.columns)}")
        
        # KSIC 컬럼명 정리
        ksic_df.columns = [str(col).strip() for col in ksic_df.columns]
        
        # 코드와 업종명 컬럼 찾기
        code_col = None
        name_col = None
        
        for col in ksic_df.columns:
            if '코드' in col or 'code' in col.lower():
                code_col = col
            if '명' in col or 'name' in col.lower() or '분류' in col:
                if name_col is None:  # 첫 번째 매칭되는 컬럼 사용
                    name_col = col
        
        print(f"   매핑 컬럼 - 코드: {code_col}, 업종명: {name_col}")
        
        # 매핑 딕셔너리 생성
        if code_col and name_col:
            # KSIC 코드를 문자열로 변환하고 정리
            ksic_df[code_col] = ksic_df[code_col].astype(str).str.strip()
            ksic_mapping_dict = dict(zip(ksic_df[code_col], ksic_df[name_col]))
            
            # 데이터프레임의 업종 코드도 문자열로 변환
            df[industry_col] = df[industry_col].astype(str).str.strip()
            
            # 업종명 매핑
            df['업종_상세명'] = df[industry_col].map(ksic_mapping_dict)
            
            # 매핑 성공 비율 확인
            mapped_count = df['업종_상세명'].notna().sum()
            total_count = len(df)
            mapping_rate = (mapped_count / total_count) * 100
            
            print(f"\n📊 매핑 결과:")
            print(f"   - 전체 기업수: {total_count:,}")
            print(f"   - 매핑 성공: {mapped_count:,} ({mapping_rate:.1f}%)")
            print(f"   - 매핑 실패: {total_count - mapped_count:,} ({100-mapping_rate:.1f}%)")
            
            # 매핑 예시 출력
            sample_mapped = df[df['업종_상세명'].notna()].sample(min(5, mapped_count))
            print("\n📋 매핑 예시:")
            for idx, row in sample_mapped.iterrows():
                print(f"   {row[industry_col]} → {row['업종_상세명']}")
        
        else:
            print("⚠️ KSIC 파일에서 적절한 코드/업종명 컬럼을 찾을 수 없습니다.")
            df['업종_상세명'] = None
            
    except Exception as e:
        print(f"⚠️ KSIC 파일 처리 중 오류: {e}")
        df['업종_상세명'] = None
    
    # 업종별 부도율 계산 (기본)
    industry_stats = df.groupby(industry_col).agg({
        target_col: ['count', 'sum', 'mean']
    }).round(4)
    
    industry_stats.columns = ['기업수', '부도기업수', '부도율']
    industry_stats['부도율(%)'] = industry_stats['부도율'] * 100
    industry_stats = industry_stats.sort_values('부도율(%)', ascending=False)
    
    print("\n" + "="*80)
    print("🏭 업종별 부도율 분석 (KSIC 업종명 포함)")
    print("="*80)
    
    # 업종명이 매핑된 경우 함께 표시
    if '업종_상세명' in df.columns and df['업종_상세명'].notna().any():
        # 업종명 정보를 industry_stats에 추가
        업종명_dict = df.groupby(industry_col)['업종_상세명'].first()
        industry_stats['업종명'] = industry_stats.index.map(업종명_dict)
        
        print("\n상위 15개 고위험 업종:")
        print("-"*80)
        print(f"{'코드':<8} {'업종명':<40} {'기업수':>8} {'부도기업':>8} {'부도율(%)':>10}")
        print("-"*80)
        
        for idx, row in industry_stats.head(15).iterrows():
            업종명 = row['업종명'] if pd.notna(row['업종명']) else '(미분류)'
            업종명 = 업종명[:38] + '..' if len(업종명) > 38 else 업종명
            print(f"{idx:<8} {업종명:<40} {row['기업수']:>8,} {row['부도기업수']:>8.0f} {row['부도율(%)']:>10.2f}")
    else:
        print("\n상위 10개 고위험 업종:")
        print(industry_stats[['기업수', '부도기업수', '부도율(%)']].head(10))
    
    # 대분류 분석
    print("\n" + "="*80)
    print("📊 업종 대분류별 부도율 분석")
    print("="*80)
    
    # 업종 코드의 첫 글자로 대분류 추출
    df['업종_대분류'] = df[industry_col].astype(str).str[0]
    
    # 대분류 설명 (한국표준산업분류 10차 기준)
    대분류_설명 = {
        'A': '농업, 임업 및 어업',
        'B': '광업',
        'C': '제조업',
        'D': '전기, 가스, 증기 및 공기조절 공급업',
        'E': '수도, 하수 및 폐기물 처리, 원료 재생업',
        'F': '건설업',
        'G': '도매 및 소매업',
        'H': '운수 및 창고업',
        'I': '숙박 및 음식점업',
        'J': '정보통신업',
        'K': '금융 및 보험업',
        'L': '부동산업',
        'M': '전문, 과학 및 기술 서비스업',
        'N': '사업시설 관리, 사업 지원 및 임대 서비스업',
        'O': '공공행정, 국방 및 사회보장 행정',
        'P': '교육 서비스업',
        'Q': '보건업 및 사회복지 서비스업',
        'R': '예술, 스포츠 및 여가관련 서비스업',
        'S': '협회 및 단체, 수리 및 기타 개인 서비스업',
        'T': '가구 내 고용활동 및 달리 분류되지 않은 자가소비 생산활동',
        'U': '국제 및 외국기관'
    }
    
    # 대분류별 통계
    major_stats = df.groupby('업종_대분류').agg({
        target_col: ['count', 'sum', 'mean'],
        '자산총계': 'median',
        '매출액': 'median'
    }).round(2)
    
    major_stats.columns = ['기업수', '부도기업수', '부도율', '자산중앙값', '매출중앙값']
    major_stats['부도율(%)'] = major_stats['부도율'] * 100
    major_stats['대분류명'] = major_stats.index.map(대분류_설명)
    major_stats = major_stats.sort_values('부도율(%)', ascending=False)
    
    print("\n대분류별 부도율 및 규모:")
    print("-"*100)
    print(f"{'대분류':<6} {'업종명':<35} {'기업수':>8} {'부도기업':>8} {'부도율(%)':>10} {'자산중앙값':>15} {'매출중앙값':>15}")
    print("-"*100)
    
    for idx, row in major_stats.head(10).iterrows():
        대분류명 = row['대분류명'] if pd.notna(row['대분류명']) else '(미분류)'
        대분류명 = 대분류명[:33] + '..' if len(대분류명) > 33 else 대분류명
        print(f"{idx:<6} {대분류명:<35} {row['기업수']:>8.0f} {row['부도기업수']:>8.0f} "
              f"{row['부도율(%)']:>10.2f} {row['자산중앙값']:>15,.0f} {row['매출중앙값']:>15,.0f}")


📊 한국표준산업분류(KSIC) 데이터 로딩 및 매핑
✅ KSIC 파일 발견. 시트 목록: ['10차개정한국표준산업분류']
✅ KSIC 데이터 로딩 완료: (1199, 10)
   컬럼: ['개정 분류체계(제10차 기준)', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']
   매핑 컬럼 - 코드: None, 업종명: 개정 분류체계(제10차 기준)
⚠️ KSIC 파일에서 적절한 코드/업종명 컬럼을 찾을 수 없습니다.

🏭 업종별 부도율 분석 (KSIC 업종명 포함)

상위 10개 고위험 업종:
          기업수  부도기업수  부도율(%)
업종(중분류)                     
B08         3      1  33.330
A01       384     15   3.910
J59       220      7   3.180
C11       127      4   3.150
I56       561     17   3.030
L68      2459     71   2.890
J63       215      6   2.790
S94        72      2   2.780
R91       463     11   2.380
C10      1917     40   2.090

📊 업종 대분류별 부도율 분석

대분류별 부도율 및 규모:
----------------------------------------------------------------------------------------------------
대분류    업종명                                      기업수     부도기업     부도율(%)           자산중앙값           매출중앙값
-----------------------------------

In [240]:
# 개별 시각화 - 2. 업종별 리스크 매핑 (부채비율 vs ROA)
if '부채비율' in industry_financial.columns and 'ROA' in industry_financial.columns:
    print("\n" + "="*60)
    print("📊 [차트 2] 업종별 리스크 매핑 - 부채비율 vs ROA")
    print("="*60)
    
    fig2 = go.Figure()
    
    # 부도율이 높은 업종만 텍스트 표시
    texts = []
    for idx in industry_financial.index:
        if industry_financial.loc[idx, target_col] > 0.02:  # 부도율 2% 이상
            texts.append(idx)
        else:
            texts.append('')
    
    fig2.add_trace(
        go.Scatter(
            x=industry_financial['부채비율'],
            y=industry_financial['ROA'],
            mode='markers+text',
            marker=dict(
                size=industry_financial[target_col]*1000,
                color=industry_financial[target_col]*100,
                colorscale='Reds',
                showscale=True,
                colorbar=dict(
                    title="부도율(%)",
                    tickformat=".2f"
                ),
                line=dict(color='white', width=1)
            ),
            text=texts,
            textposition='top center',
            textfont=dict(size=10),
            hovertemplate='업종: %{hovertext}<br>부채비율: %{x:.0f}%<br>ROA: %{y:.1f}%<br>부도율: %{customdata:.2f}%<extra></extra>',
            hovertext=industry_financial.index,
            customdata=industry_financial[target_col]*100,
            showlegend=False
        )
    )
    
    # 참조선 추가
    fig2.add_hline(y=0, line_dash="dash", line_color="gray", line_width=1, 
                   annotation_text="ROA = 0 (손익분기점)")
    fig2.add_vline(x=200, line_dash="dash", line_color="gray", line_width=1,
                   annotation_text="부채비율 200% (경고선)")
    
    # 위험 구역 표시
    fig2.add_shape(
        type="rect",
        x0=200, x1=industry_financial['부채비율'].max(),
        y0=industry_financial['ROA'].min(), y1=0,
        fillcolor="red", opacity=0.1,
        line=dict(width=0)
    )
    
    fig2.update_layout(
        title='업종별 리스크 매핑 (버블 크기 = 부도율)',
        xaxis_title='부채비율 (%, 중앙값)',
        yaxis_title='ROA (%, 중앙값)',
        height=600,
        width=900,
        margin=dict(l=80, r=150, t=80, b=80),
        annotations=[
            dict(
                x=industry_financial['부채비율'].max() * 0.9,
                y=industry_financial['ROA'].min() * 0.9,
                text="고위험 구역<br>(높은 부채 + 낮은 수익성)",
                showarrow=False,
                font=dict(color="red", size=12),
                bgcolor="white",
                bordercolor="red",
                borderwidth=1
            )
        ]
    )
    
    fig2.show()
    
    # 위험 구역 업종 리스트
    high_risk_zone = industry_financial[
        (industry_financial['부채비율'] > 200) & 
        (industry_financial['ROA'] < 0)
    ]
    if len(high_risk_zone) > 0:
        print("\n⚠️ 고위험 구역 업종 (부채비율 > 200% & ROA < 0):")
        for idx in high_risk_zone.index:
            print(f"  • {idx}: 부채비율 {high_risk_zone.loc[idx, '부채비율']:.0f}%, "
                  f"ROA {high_risk_zone.loc[idx, 'ROA']:.1f}%, "
                  f"부도율 {high_risk_zone.loc[idx, target_col]*100:.2f}%")


📊 [차트 2] 업종별 리스크 매핑 - 부채비율 vs ROA


In [241]:
# 개별 시각화 - 3. 업종별 유동성 vs 부도율
if '유동비율' in industry_financial.columns:
    print("\n" + "="*60)
    print("📊 [차트 3] 업종별 유동성 분석 - 유동비율 vs 부도율")
    print("="*60)
    
    fig3 = go.Figure()
    
    # 유동성 위기 업종 식별
    liquidity_crisis = industry_financial['유동비율'] < 1.0
    
    # 색상 설정
    colors = ['red' if crisis else 'blue' for crisis in liquidity_crisis]
    symbols = ['diamond' if crisis else 'circle' for crisis in liquidity_crisis]
    
    fig3.add_trace(
        go.Scatter(
            x=industry_financial['유동비율'],
            y=industry_financial[target_col]*100,
            mode='markers+text',
            marker=dict(
                size=industry_financial['기업수']/30,  # 버블 크기 조정
                color=industry_financial['현금비율'],
                colorscale='Blues',
                showscale=True,
                colorbar=dict(
                    title="현금비율",
                    tickformat=".3f"
                ),
                symbol=symbols,
                line=dict(color='darkblue', width=1)
            ),
            text=[idx if industry_financial.loc[idx, '유동비율'] < 1.0 or 
                      industry_financial.loc[idx, target_col] > 0.025 
                  else '' for idx in industry_financial.index],
            textposition='top center',
            textfont=dict(size=9),
            hovertemplate='업종: %{hovertext}<br>유동비율: %{x:.2f}<br>부도율: %{y:.2f}%<br>기업수: %{customdata}<extra></extra>',
            hovertext=industry_financial.index,
            customdata=industry_financial['기업수'],
            showlegend=False
        )
    )
    
    # 위험선 추가
    fig3.add_vline(x=1.0, line_dash="dash", line_color="red", line_width=2,
                   annotation_text="유동비율 1.0 (위험선)")
    fig3.add_hline(y=df[target_col].mean()*100, line_dash="dot", line_color="green", 
                   line_width=1,
                   annotation_text=f"전체 평균 부도율 ({df[target_col].mean()*100:.2f}%)")
    
    # 위험 구역 표시
    fig3.add_shape(
        type="rect",
        x0=0, x1=1,
        y0=0, y1=industry_financial[target_col].max()*100,
        fillcolor="red", opacity=0.1,
        line=dict(width=0)
    )
    
    fig3.update_layout(
        title='업종별 유동성 vs 부도율 (버블 크기 = 기업수, 색상 = 현금비율)',
        xaxis_title='유동비율 (중앙값)',
        yaxis_title='부도율 (%)',
        height=600,
        width=900,
        margin=dict(l=80, r=150, t=80, b=80),
        xaxis=dict(range=[0, max(3, industry_financial['유동비율'].max()*1.1)]),
        annotations=[
            dict(
                x=0.5,
                y=industry_financial[target_col].max()*100 * 0.9,
                text="유동성 위기<br>구역",
                showarrow=False,
                font=dict(color="red", size=11),
                bgcolor="white",
                bordercolor="red",
                borderwidth=1
            )
        ]
    )
    
    fig3.show()
    
    # 유동성 위기 업종 상세
    crisis_industries = industry_financial[industry_financial['유동비율'] < 1.0].sort_values(target_col, ascending=False)
    if len(crisis_industries) > 0:
        print("\n💧 유동성 위기 업종 (유동비율 < 1.0):")
        print(f"{'업종':<6} {'유동비율':>8} {'현금비율':>8} {'부도율(%)':>10} {'기업수':>8}")
        print("-" * 50)
        for idx in crisis_industries.index:
            print(f"{idx:<6} {crisis_industries.loc[idx, '유동비율']:>8.2f} "
                  f"{crisis_industries.loc[idx, '현금비율']:>8.3f} "
                  f"{crisis_industries.loc[idx, target_col]*100:>10.2f} "
                  f"{crisis_industries.loc[idx, '기업수']:>8.0f}")


📊 [차트 3] 업종별 유동성 분석 - 유동비율 vs 부도율



💧 유동성 위기 업종 (유동비율 < 1.0):
업종         유동비율     현금비율     부도율(%)      기업수
--------------------------------------------------
R91        0.61    0.030       2.00      463
I55        0.32    0.010       1.00      276


In [242]:
# 개별 시각화 - 4. 업종 규모 vs 부도율
if '기업수' in industry_financial.columns:
    print("\n" + "="*60)
    print("📊 [차트 4] 업종 규모와 부도율 관계")
    print("="*60)
    
    fig4 = go.Figure()
    
    # 주요 업종만 라벨 표시 (기업수 1000개 이상 또는 부도율 2.5% 이상)
    texts = []
    for idx in industry_financial.index:
        if industry_financial.loc[idx, '기업수'] > 1000 or industry_financial.loc[idx, target_col] > 0.025:
            texts.append(idx)
        else:
            texts.append('')
    
    fig4.add_trace(
        go.Scatter(
            x=industry_financial['기업수'],
            y=industry_financial[target_col]*100,
            mode='markers+text',
            marker=dict(
                size=industry_financial['기업수']/50,  # 버블 크기
                color=industry_financial['부채비율'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(
                    title="부채비율(%)",
                    tickformat=".0f"
                ),
                line=dict(color='darkgreen', width=1)
            ),
            text=texts,
            textposition='top center',
            textfont=dict(size=10),
            hovertemplate='업종: %{hovertext}<br>기업수: %{x:,}<br>부도율: %{y:.2f}%<br>부채비율: %{customdata:.0f}%<extra></extra>',
            hovertext=industry_financial.index,
            customdata=industry_financial['부채비율'],
            showlegend=False
        )
    )
    
    # 평균 부도율 선
    avg_bankruptcy = df[target_col].mean() * 100
    fig4.add_hline(y=avg_bankruptcy, line_dash="dash", line_color="red", 
                   line_width=1,
                   annotation_text=f"전체 평균 부도율 ({avg_bankruptcy:.2f}%)")
    
    # 사분면 구분을 위한 중앙값 선
    median_count = industry_financial['기업수'].median()
    fig4.add_vline(x=median_count, line_dash="dot", line_color="gray", 
                   line_width=1, opacity=0.5,
                   annotation_text=f"기업수 중앙값 ({median_count:.0f})")
    
    # 사분면 레이블 추가
    fig4.add_annotation(
        x=industry_financial['기업수'].max() * 0.8,
        y=industry_financial[target_col].max() * 100 * 0.9,
        text="대규모<br>고위험",
        showarrow=False,
        font=dict(color="red", size=12),
        bgcolor="white",
        bordercolor="red",
        borderwidth=1
    )
    
    fig4.add_annotation(
        x=industry_financial['기업수'].min() * 1.5,
        y=industry_financial[target_col].max() * 100 * 0.9,
        text="소규모<br>고위험",
        showarrow=False,
        font=dict(color="orange", size=12),
        bgcolor="white",
        bordercolor="orange",
        borderwidth=1
    )
    
    fig4.update_layout(
        title='업종 규모 vs 부도율 (버블 크기 = 기업수, 색상 = 부채비율)',
        xaxis_title='업종별 기업 수',
        yaxis_title='부도율 (%)',
        height=600,
        width=900,
        margin=dict(l=80, r=150, t=80, b=80),
        xaxis_type="log"  # 로그 스케일로 표시
    )
    
    fig4.show()
    
    # 규모별 업종 분석
    large_high_risk = industry_financial[
        (industry_financial['기업수'] > median_count) & 
        (industry_financial[target_col] > df[target_col].mean())
    ].sort_values(target_col, ascending=False)
    
    if len(large_high_risk) > 0:
        print("\n🔴 대규모-고위험 업종 (많은 기업 + 높은 부도율):")
        print(f"{'업종':<6} {'기업수':>8} {'부도율(%)':>10} {'부채비율':>10}")
        print("-" * 40)
        for idx in large_high_risk.head(5).index:
            print(f"{idx:<6} {large_high_risk.loc[idx, '기업수']:>8.0f} "
                  f"{large_high_risk.loc[idx, target_col]*100:>10.2f} "
                  f"{large_high_risk.loc[idx, '부채비율']:>10.0f}")


📊 [차트 4] 업종 규모와 부도율 관계



🔴 대규모-고위험 업종 (많은 기업 + 높은 부도율):
업종          기업수     부도율(%)       부채비율
----------------------------------------
I56         561       3.00        112
L68        2459       3.00        143
C10        1917       2.00        165
C13         431       2.00        131
C14         396       2.00        127


In [243]:
# 개별 시각화 - 5. 업종별 종합 히트맵
if len(industry_financial) > 0:
    print("\n" + "="*60)
    print("📊 [차트 5] 업종별 주요 지표 히트맵")
    print("="*60)
    
    # 상위 20개 업종만 선택 (부도율 기준)
    top20_industries = industry_financial.nlargest(20, target_col)
    
    # 히트맵용 데이터 준비
    heatmap_data = pd.DataFrame({
        '부도율': top20_industries[target_col] * 100,
        '부채비율': top20_industries['부채비율'],
        'ROA': top20_industries['ROA'],
        '유동비율': top20_industries['유동비율'],
        '현금비율': top20_industries['현금비율'] * 100  # 퍼센트로 변환
    })
    
    # NaN 값 처리
    heatmap_data = heatmap_data.fillna(0)
    
    # 정규화 (0-1 스케일) - 각 컬럼별로 수동 정규화
    heatmap_normalized = pd.DataFrame(index=heatmap_data.index)
    
    for col in heatmap_data.columns:
        col_data = heatmap_data[col]
        min_val = col_data.min()
        max_val = col_data.max()
        
        # min과 max가 같은 경우 처리
        if max_val - min_val == 0:
            heatmap_normalized[col] = 0.5  # 중간값으로 설정
        else:
            heatmap_normalized[col] = (col_data - min_val) / (max_val - min_val)
    
    # Plotly 히트맵
    fig5 = go.Figure(data=go.Heatmap(
        z=heatmap_normalized.values,
        x=heatmap_normalized.columns,
        y=heatmap_normalized.index,
        colorscale='RdYlBu_r',  # 빨강(위험) - 파랑(안전)
        showscale=True,
        colorbar=dict(
            title="정규화 값<br>(0=최소, 1=최대)",
            thickness=20,
            len=0.7
        ),
        text=heatmap_data.round(2).values,
        texttemplate='%{text}',
        textfont={"size": 10},
        hovertemplate='업종: %{y}<br>지표: %{x}<br>실제값: %{text}<br>정규화값: %{z:.2f}<extra></extra>'
    ))
    
    fig5.update_layout(
        title='업종별 주요 지표 히트맵 (상위 20개 고위험 업종)',
        xaxis_title='재무 지표',
        yaxis_title='업종 코드',
        height=700,
        width=900,
        margin=dict(l=100, r=100, t=80, b=80),
        xaxis={'side': 'bottom'},
        yaxis={'side': 'left'}
    )
    
    # 축 라벨 회전
    fig5.update_xaxes(tickangle=45)
    
    fig5.show()
    
    # 위험 점수 계산 및 순위
    print("\n📊 업종별 종합 위험 점수 (상위 10개)")
    print("-" * 60)
    
    # 위험 점수 계산 (높을수록 위험)
    # 각 지표별 위험 기여도 계산
    risk_components = pd.DataFrame(index=heatmap_data.index)
    
    # 부도율: 높을수록 위험 (가중치 30%)
    risk_components['부도율_점수'] = heatmap_normalized['부도율'] * 30
    
    # 부채비율: 높을수록 위험 (가중치 20%)
    risk_components['부채비율_점수'] = heatmap_normalized['부채비율'] * 20
    
    # ROA: 낮을수록 위험 (가중치 20%)
    risk_components['ROA_점수'] = (1 - heatmap_normalized['ROA']) * 20
    
    # 유동비율: 낮을수록 위험 (가중치 15%)
    risk_components['유동비율_점수'] = (1 - heatmap_normalized['유동비율']) * 15
    
    # 현금비율: 낮을수록 위험 (가중치 15%)
    risk_components['현금비율_점수'] = (1 - heatmap_normalized['현금비율']) * 15
    
    # 총 위험점수 계산
    risk_score = risk_components.sum(axis=1)
    
    # 위험 점수와 함께 출력
    risk_df = pd.DataFrame({
        '위험점수': risk_score.round(1),
        '부도율(%)': heatmap_data['부도율'].round(2),
        '부채비율': heatmap_data['부채비율'].round(0),
        'ROA': heatmap_data['ROA'].round(1),
        '유동비율': heatmap_data['유동비율'].round(2),
        '현금비율(%)': heatmap_data['현금비율'].round(2),
        '기업수': top20_industries['기업수']
    }).sort_values('위험점수', ascending=False)
    
    print(risk_df.head(10).to_string())
    
    # 위험 등급 분류
    print("\n🎯 위험 등급별 업종 분류")
    print("-" * 60)
    
    def get_risk_level(score):
        if score >= 70:
            return '🔴 매우위험'
        elif score >= 50:
            return '🟠 위험'
        elif score >= 30:
            return '🟡 주의'
        else:
            return '🟢 안전'
    
    risk_df['위험등급'] = risk_df['위험점수'].apply(get_risk_level)
    
    # 등급별 업종 수
    risk_level_counts = risk_df['위험등급'].value_counts()
    for level in ['🔴 매우위험', '🟠 위험', '🟡 주의', '🟢 안전']:
        if level in risk_level_counts.index:
            count = risk_level_counts[level]
            print(f"{level}: {count}개 업종")
            
            # 해당 등급의 업종 리스트 (최대 3개)
            level_industries = risk_df[risk_df['위험등급'] == level].head(3)
            for idx, row in level_industries.iterrows():
                print(f"    - {idx}: 위험점수 {row['위험점수']:.1f}, 부도율 {row['부도율(%)']}%")
    
    # 디버깅 정보
    print("\n📝 위험점수 계산 검증")
    print("-" * 60)
    print("정규화된 값 범위 확인:")
    for col in heatmap_normalized.columns:
        print(f"  {col}: Min={heatmap_normalized[col].min():.3f}, Max={heatmap_normalized[col].max():.3f}")
    
    print("\n상위 3개 업종 상세 점수:")
    for idx in risk_df.head(3).index:
        print(f"\n{idx}:")
        print(f"  부도율 기여: {risk_components.loc[idx, '부도율_점수']:.1f}")
        print(f"  부채비율 기여: {risk_components.loc[idx, '부채비율_점수']:.1f}")
        print(f"  ROA 기여: {risk_components.loc[idx, 'ROA_점수']:.1f}")
        print(f"  유동비율 기여: {risk_components.loc[idx, '유동비율_점수']:.1f}")
        print(f"  현금비율 기여: {risk_components.loc[idx, '현금비율_점수']:.1f}")
        print(f"  총 위험점수: {risk_score[idx]:.1f}")


📊 [차트 5] 업종별 주요 지표 히트맵



📊 업종별 종합 위험 점수 (상위 10개)
------------------------------------------------------------
          위험점수  부도율(%)    부채비율    ROA  유동비율  현금비율(%)   기업수
업종(중분류)                                                   
A01     85.200   4.000 152.000  1.900 1.440    0.000   384
I56     72.700   3.000 112.000  0.300 1.100    0.000   561
L68     72.200   3.000 143.000 -0.200 1.640    2.000  2459
J59     66.200   3.000  98.000  0.600 1.440    0.000   220
C31     58.300   2.000 152.000  1.500 1.310    0.000   369
J63     56.900   3.000  92.000  2.200 1.530    0.000   215
C11     54.700   3.000 101.000  0.700 1.280    5.000   127
S94     53.900   3.000  70.000  1.600 1.790    0.000    72
C10     53.500   2.000 165.000  2.500 1.670    0.000  1917
C17     50.200   2.000 150.000  2.800 1.480    0.000   335

🎯 위험 등급별 업종 분류
------------------------------------------------------------
🔴 매우위험: 3개 업종
    - A01: 위험점수 85.2, 부도율 4.0%
    - I56: 위험점수 72.7, 부도율 3.0%
    - L68: 위험점수 72.2, 부도율 3.0%
🟠 위험: 7개 업종
    - J59:

### 4.2 규모별 부도 패턴

In [244]:
# 기업 규모 관련 분석
size_indicators = ['자산총계', '매출액', '종업원수']
existing_size_cols = [col for col in size_indicators if col in df.columns]

if existing_size_cols and target_col in df.columns:
    for size_col in existing_size_cols:
        if df[size_col].notna().sum() > 0:
            # 규모별 구간 생성 (분위수 기준)
            quantiles = df[size_col].quantile([0.25, 0.5, 0.75])
            
            bins = [-np.inf, quantiles[0.25], quantiles[0.5], quantiles[0.75], np.inf]
            labels = ['소규모(하위25%)', '중소규모(25-50%)', '중규모(50-75%)', '대규모(상위25%)']
            
            df[f'{size_col}_규모'] = pd.cut(df[size_col], bins=bins, labels=labels)
            
            # 규모별 부도율
            size_bankruptcy = df.groupby(f'{size_col}_규모')[target_col].agg(['count', 'sum', 'mean'])
            size_bankruptcy.columns = ['기업수', '부도기업수', '부도율']
            size_bankruptcy['부도율(%)'] = size_bankruptcy['부도율'] * 100
            
            print(f"\n📏 {size_col} 기준 규모별 부도율")
            print("="*60)
            print(size_bankruptcy)
            
            # 시각화
            fig = go.Figure()
            fig.add_trace(go.Bar(
                x=size_bankruptcy.index,
                y=size_bankruptcy['부도율(%)'],
                text=[f"{v:.2f}%" for v in size_bankruptcy['부도율(%)']],
                textposition='outside',
                marker_color=['lightblue', 'blue', 'darkblue', 'navy']
            ))
            
            fig.update_layout(
                title=f'{size_col} 기준 기업 규모별 부도율',
                xaxis_title='기업 규모',
                yaxis_title='부도율 (%)',
                showlegend=False
            )
            fig.show()
            break


📏 자산총계 기준 규모별 부도율
                기업수  부도기업수   부도율  부도율(%)
자산총계_규모                                 
소규모(하위25%)    12527    205 0.016   1.636
중소규모(25-50%)  12526    209 0.017   1.669
중규모(50-75%)   12526    201 0.016   1.605
대규모(상위25%)    12526    144 0.011   1.150


### 4.3 외감 여부별 부도 패턴

In [245]:
# 외감 여부 분석
audit_cols = [col for col in df.columns if '외감' in col]

if audit_cols and target_col in df.columns:
    audit_col = audit_cols[0]
    
    # 외감 여부별 부도율
    audit_bankruptcy = pd.crosstab(df[audit_col], df[target_col], 
                                  normalize='index') * 100
    
    print("\n📋 외감 여부별 부도율 분석")
    print("="*60)
    print(audit_bankruptcy)
    
    # 외감 여부별 주요 재무지표 비교
    if '부채비율' in df.columns and 'ROA' in df.columns:
        audit_comparison = df.groupby(audit_col).agg({
            '부채비율': ['mean', 'median'],
            'ROA': ['mean', 'median'],
            target_col: 'mean'
        }).round(2)
        
        print("\n외감 여부별 재무지표 비교")
        print(audit_comparison)


📋 외감 여부별 부도율 분석
모형개발용Performance(향후1년내부도여부)      0     1
외감구분                                    
1                           98.637 1.363
2                           98.399 1.601

외감 여부별 재무지표 비교
        부채비율           ROA        모형개발용Performance(향후1년내부도여부)
        mean  median  mean median                        mean
외감구분                                                         
1    277.210 109.080 2.410  2.140                       0.010
2    -78.950 115.210 2.230  3.130                       0.020
